# 使用 pythonnet 的 clr 模組操作 群益 SKCOMAPI

## 前置工作
### 1. 轉換 SKCOM.dll 成 type library
### 2. 安裝 pythonnet

### 1.用 tlbimp.exe 將 SKCOM.dll 轉成  Type library
    我是安裝 visual studio 2022，利用它附帶的 comandline prompt terminal ，下指令 tlbimp.exe 將 SKCOM.dll 轉成 typelib 形式，最終產生 SKCOMLib.dll 的檔案。之後就可以輕鬆用 pythonnet 的 clr 模組取用。

![圖](vs_2022_convert_dll_to_tlb.png)

### 2. 安裝 pythonnet
    Python.NET 模組可以讓 python 使用者無縫整合 .NET Common Language Runtime (CLR)。它讓 Python 可以跟 CLR 互動, 甚至也可以反過來將 Python 包進 .NET 裡.
    在你的 python 環境下用 pip 安裝 pythonnet 即可。詳細操作請看 pythonnet 官網

In [23]:
!pip install pythonnet

更新後的OnTimer 104957
更新後的OnTimer 105057
更新後的OnTimer 105157
更新後的OnTimer 105257
更新後的OnTimer 105357
更新後的OnTimer 105457
更新後的OnTimer 105557
更新後的OnTimer 105657
更新後的OnTimer 105757
更新後的OnTimer 105857
更新後的OnTimer 105957


# clr 模組操作 群益 SKCOMAPI 操作範例

In [ ]:
import clr
# AddReference，以下兩種可以擇一使用，都不用加副檔名 .dll
#clr.AddReference("SKCOMLib") # SKCOMLib.dll 與主程式同目錄
clr.AddReference(r"C:\skcom\CapitalAPI_2.13.41\x64\SKCOMLib") #使用絕對路徑

# 加入參考後，就可以用下列方式引用
from SKCOMLib import SKCenterLib, SKReplyLib, SKQuoteLib, SKOrderLib, SKSTOCKLONG


## 建立Event loop

#### 1. 使用 jupyterlab magic function
#### 2. 使用 PumpWaiteMessage
#### 3. 建立GUI介面，有內建的 eventloop (本範例未使用)

#### 方法1. 使用 jupyter magic function
我使用的是 jupyterlab，使用下列方式即可產生自動 event loop，可以輕鬆即時互動式修改，很方便。大部分情況都可以這樣使用，若無反應，則使用方法2

In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


#### 方法2. 使用 PumpWaiteMessage
要使用 pythoncom 的 PumpWaiteMessage， 而 pythoncom 需要先安裝 pywin32。我這邊示範用 asyncio 來實作，好處也是可以即時互動修改，避免因為持續 pooling 而無法繼續。

In [3]:
# 如果沒有pywin32, 請先用 pip 安裝，下面是 jupyter magic func
!pip install pywin32

In [4]:
import asyncio
from pythoncom import PumpWaitingMessages

# working functions, async coruntime to pump events
async def pump_task():
    while True:
        PumpWaitingMessages()
        await asyncio.sleep(0.01)

# get an event loop 
loop = asyncio.get_event_loop()
# 將 pump coruntime 加入列隊
pumping_loop = loop.create_task(pump_task())
print('Pumping events!')


Pumping events!


## 建立 SKCOMAPI元件, event callback, 與綁定各自 event callback

In [5]:
# 建立 skcomapi 元件
skC = SKCenterLib()
skR = SKReplyLib()
skQ = SKQuoteLib()
skO = SKOrderLib()

In [6]:
# database, 存取資料用，可以使用任容器/database來裝，這裡示範的是用python的 list
tickdata = []
quotedata = []

## 建立 event callback, 根據 api 選用你想要使用的 event callback

In [7]:
# skC 相關
def OnTimer(nTime):
    """只要Login後就會每分鐘持續回傳伺服器時間，可以藉此觀察 event loop 是否有成功"""
    print("伺服器時間", nTime)


# skR 相關
def OnReplyMessage(bstrUserID , bstrMessage, sConfirmCode):
    """ 這個一定要建立，Login 時會檢查 sConfirmCode 是否 == -1"""
    sConfirmCode = -1
    print("OnReplyMessage", bstrMessage)
    return sConfirmCode


def OnConnect(bstrUserID, nErrorCode  ):
    """回報群益的回報伺服器連線狀況"""
    print('SKR_OnConnect', bstrUserID, nErrorCode)


# skQ 相關
def OnConnection(nKind, nCode):
    """回報連線報價伺服器狀態
    nKind: 3001 連線報價伺服器
           3002 離線報價伺服器
           3003 連線成功，收到3003後，始可登錄報價商品
    """
    print('skQ_OnConnection', nKind, nCode)


def OnNotifyQuoteLONG(sMarketNo, nIndex):
    ts = SKSTOCKLONG()
    ncode, ts = skQ.SKQuoteLib_GetStockByIndexLONG(sMarketNo=sMarketNo, nStockIdx= nIndex, pSKStockLONG=ts)
    quotedata.append([ts.bstrStockName, ts.nClose/ 10**ts.sDecimal, ts.nTickQty, ts.nTQty, ts.nYQty])
    print(quotedata[-1])


def OnNotifyTicksLONG(sMarketNo, nIndex, nPtr, nDate, nTimehms, nTimemillismicros, nBid, nAsk, nClose, nQty, nSimulate):
    ts = SKSTOCKLONG()
    ncode, ts = skQ.SKQuoteLib_GetStockByIndexLONG(sMarketNo, nIndex, ts)
    tickdata.append([ts.bstrStockName, nTimehms, nBid/10**ts.sDecimal, nAsk/10**ts.sDecimal, nClose/10**ts.sDecimal, nQty])
    print(f"{ts.bstrStockName} time: {nTimehms} bid= {nBid/10**ts.sDecimal}, ask= {nAsk/10**ts.sDecimal}, close= {nClose/10**ts.sDecimal}, qty= {nQty}")


def OnNotifyKLineData(bstrStockNo, bstrData):
    print(bstrStockNo, bstrData)

## 使用 += (綁定) 或 -= (解除綁定) 各個 event callback

In [8]:
# 綁定 event callback
# skC相關 event
skC.OnTimer += OnTimer

# skR相關 event
skR.OnConnect += OnConnect
skR.OnReplyMessage += OnReplyMessage

# skQ相關 event
skQ.OnConnection += OnConnection
skQ.OnNotifyQuoteLONG += OnNotifyQuoteLONG
skQ.OnNotifyTicksLONG += OnNotifyTicksLONG
skQ.OnNotifyKLineData += OnNotifyKLineData

In [9]:
# 解除 callback 綁定
# 注意如果想在 jupyter 裡邊運行邊調整 callback 
# 請先解除綁定 callback，然後再重新綁定
skC.OnTimer -= OnTimer

# 更新 OnTimer callback
def OnTimer(nTime):
    print("更新後的OnTimer", nTime)

# 重新綁定 Event callback
skC.OnTimer += OnTimer


# Login

In [10]:
ID = ""
PW = ""
ncode = skC.SKCenterLib_Login(ID,PW)
print("Login", skC.SKCenterLib_GetReturnCodeMessage(ncode))

OnReplyMessage SKReplyLib_OnReplyMessage:Announcement callback.
Login SK_SUCCESS


# 連線報價伺服器

In [15]:
# Enter Monitor
#ncode = skQ.SKQuoteLib_LeaveMonitor()
ncode = skQ.SKQuoteLib_EnterMonitorLONG()
print("EnterMonitor", skC.SKCenterLib_GetReturnCodeMessage(ncode))

EnterMonitor SK_SUCCESS
skQ_OnConnection 3001 0
skQ_OnConnection 3003 0


# 報價
## 等到 OnConnection 出現 3003，才可以 requestStock/Ticks

In [12]:
strStock = "1101"
page, ncode = skQ.SKQuoteLib_RequestStocks(1, strStock) # 會返回 page跟 ncode會返回 page跟 ncode
print(f"RequestStocks= {strStock} ", skC.SKCenterLib_GetReturnCodeMessage(ncode))

RequestStocks= 1101  1
['台泥', 0.0, 0, 0, 14481]
['台泥', 37.55, 5, 11103, 14481]


In [16]:
strStock = "MTX00"
ncode, page = skQ.SKQuoteLib_RequestTicks(1, strStock) # 會返回 page跟 ncode會返回 page跟 ncode
print(f"RequestStocks= {strStock} ", skC.SKCenterLib_GetReturnCodeMessage(ncode))

RequestStocks= MTX00  SK_SUCCESS
小台近 time: 102835 bid= 15675.0, ask= 15676.0, close= 15675.0, qty= 1
小台近 time: 102836 bid= 15675.0, ask= 15676.0, close= 15675.0, qty= 23
小台近 time: 102836 bid= 15675.0, ask= 15676.0, close= 15674.0, qty= 2
小台近 time: 102836 bid= 15675.0, ask= 15676.0, close= 15674.0, qty= 1
小台近 time: 102836 bid= 15675.0, ask= 15676.0, close= 15674.0, qty= 1
小台近 time: 102836 bid= 15675.0, ask= 15676.0, close= 15675.0, qty= 1
小台近 time: 102836 bid= 15675.0, ask= 15676.0, close= 15674.0, qty= 1
小台近 time: 102838 bid= 15674.0, ask= 15675.0, close= 15675.0, qty= 1
小台近 time: 102838 bid= 15674.0, ask= 15675.0, close= 15674.0, qty= 1
小台近 time: 102838 bid= 15674.0, ask= 15675.0, close= 15674.0, qty= 1
小台近 time: 102838 bid= 15674.0, ask= 15675.0, close= 15675.0, qty= 1
小台近 time: 102838 bid= 15674.0, ask= 15675.0, close= 15675.0, qty= 1
小台近 time: 102839 bid= 15674.0, ask= 15675.0, close= 15675.0, qty= 1
小台近 time: 102839 bid= 15674.0, ask= 15675.0, close= 15674.0, qty= 3
小台近 time: 1028

In [17]:
# 離開報價伺服器
skQ.SKQuoteLib_LeaveMonitor()

小台近 time: 102847 bid= 15672.0, ask= 15673.0, close= 15673.0, qty= 1


0

skQ_OnConnection 3002 0


In [19]:
# 存取 quotedata
quotedata

[['台泥', 0.0, 0, 0, 14481],
 ['台泥', 37.55, 5, 11103, 14481],
 ['台泥', 37.55, 5, 11103, 14481]]

更新後的OnTimer 102857


In [20]:
# 存取 tickdata
tickdata[0:5]

[['小台近', 102835, 15675.0, 15676.0, 15675.0, 1],
 ['小台近', 102836, 15675.0, 15676.0, 15675.0, 23],
 ['小台近', 102836, 15675.0, 15676.0, 15674.0, 2],
 ['小台近', 102836, 15675.0, 15676.0, 15674.0, 1],
 ['小台近', 102836, 15675.0, 15676.0, 15674.0, 1]]

更新後的OnTimer 102957
更新後的OnTimer 103057
更新後的OnTimer 103157
更新後的OnTimer 103257
更新後的OnTimer 103357
更新後的OnTimer 103457
更新後的OnTimer 103557
